# Import the needed libraries

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import folium
from shapely import geometry
import matplotlib.pyplot as plt

%matplotlib inline

# Red in the Perigrines .csv file

In [4]:
Perigrines = pd.read_csv(r'C:\Users\emack\Documents\UAF-Temperature\data\pfunique2.csv') # save to a dataframe                   
Perigrines.head() # This shows the first 5 entries of the DF.


,longitude,latitude
0,48.2282,-122.3490
1,44.5620,-68.8043
2,40.7545,-73.9645
3,40.4393,-73.9869
4,41.4992,-81.7147


# Confirm the correct CRS
From the metadata we have that the geophysics data is referenced to latlong (NAD27) datum. Let's use pyproj to transform it to both latlong (WGS84) and UTM zone 16 (WGS84).

In [5]:
# North American Datum 1927
p1 = pyproj.Proj(proj='latlong', datum='NAD27')
# WGS84 Latlong
p2 = pyproj.Proj(proj='latlong', datum='WGS84')
# WGS84 UTM Zone 16
p3 = pyproj.Proj(proj='utm', zone=16, datum='WGS84')
Perigrines['long_wgs84'], Perigrines['lat_wgs84'] = pyproj.transform(p1, p2, 
                                                            Perigrines.longitude.values, 
                                                            Perigrines.latitude.values)
Perigrines['E_utm'], Perigrines['N_utm'] = pyproj.transform(p1, p3,
                                                        Perigrines.longitude.values,
                                                        Perigrines.latitude.values)

# Convert our pandas DataFrame into a geopandas GeoDataFrame

In [6]:
Perigrines['geometry'] = [geometry.Point(x, y) for x, y in zip(Perigrines['long_wgs84'], Perigrines['lat_wgs84'])]

### Pass the DataFrame to a GeoDataFrame object indicating which one is the geometry columns and the CRS (Coordinate Reference System):

In [7]:
Perigrines = gpd.GeoDataFrame(Perigrines, geometry='geometry', crs="+init=epsg:4326")

# Save the file for re-use when we start working with other layers of data

In [8]:
Perigrines.to_csv(r'C:\Users\emack\Documents\UAF-Temperature\data\mag.csv')

# Verify our new file looks ok

In [9]:
Perigrines.head(3)

,longitude,latitude,long_wgs84,lat_wgs84,E_utm,N_utm,geometry
0,48.2282,-122.3490,48.2282,-122.3490,inf,inf,POINT (48.228 -122.349)
1,44.5620,-68.8043,44.5620,-68.8043,2.273971e+06,-1.160805e+07,POINT (44.562 -68.804)
2,40.7545,-73.9645,40.7545,-73.9645,1.919624e+06,-1.111200e+07,POINT (40.755 -73.965)


### .envelope is a shapely method to get the bounding rectangle of all the points in the geometry

In [10]:
multipoints = geometry.MultiPoint(Perigrines['geometry'])
bounds = multipoints.envelope

### Let's save the bounds for the next parts of Integrating & Exploring.

In [11]:
gpd.GeoSeries(bounds).to_file(r'C:\Users\emack\Documents\UAF-Temperature\data\area_of_study_bounds.gpkg', 'GPKG')

### Let's get the coordinates of the .boundary of the bounds.

In [12]:
coords = np.vstack(bounds.boundary.coords.xy)

### Calculate the mean of the coordinates using numpy to center the location map on this.

In [13]:
UAF_coords = (64.8558, -147.8335)
map_center = list(coords.mean(1))[::-1] # use a slice to get all the rows from the last column 

In [14]:
UAF_coords

(64.8558, -147.8335)

# Anecdotaly zoom into Fairbanks, Alaska

In [15]:
m = folium.Map(location=UAF_coords, zoom_start=8, control_scale=True)
folium.PolyLine(coords[::-1].T).add_to(m)
folium.LatLngPopup().add_to(m)
m

In [29]:
Perigrines.latitude

0       -122.3490
1        -68.8043
2        -73.9645
3        -73.9869
4        -81.7147
           ...   
24243    -70.9424
24244   -118.0520
24245   -119.5450
24246    -99.2450
24247    -77.6690
Name: latitude, Length: 24248, dtype: float64

In [30]:
Perigrines.longitude

0        48.2282
1        44.5620
2        40.7545
3        40.4393
4        41.4992
          ...   
24243    42.5297
24244    34.0264
24245    49.3100
24246    19.3897
24247    42.9002
Name: longitude, Length: 24248, dtype: float64